In [10]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
import base64
import os
import hashlib

def generate_key():
    # Generate a random 256-bit (32-byte) AES key
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        iterations=100000,
        salt=salt,
        length=32  # Use 32 for a 256-bit key
    )
    key = kdf.derive(b'YourSuperSecretPassphrase')
    return key

def encrypt(plaintext, key):
    # Generate a random 128-bit (16-byte) IV (Initialization Vector)
    iv = os.urandom(16)

    # Create an AES cipher with the key and IV
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv))
    encryptor = cipher.encryptor()

    # Encrypt the plaintext
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    # Return the IV and ciphertext
    return iv + ciphertext

def decrypt(ciphertext, key):
    # Split the IV and ciphertext
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]

    # Create an AES cipher with the key and IV
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv))
    decryptor = cipher.decryptor()

    # Decrypt the ciphertext
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    return plaintext

def dh_key_exchange():
    l = []
    # while True:
    #     x1 = int(input("User 1 - Enter Your Private Key: "))
    #     if x1 >= P:
    #         print(f"Private Key should be less than {P}")
    #     else:
    #         break

    # while True:
    #     x2 = int(input("User 2 - Enter Your Private Key: "))
    #     if x2 >= P:
    #         print(f"Private Key should be less than {P}")
    #     else:
    #         break

    P = 70  # A large prime number
    G = 23
    x1=10
    x2=6

    # Calculate Public Keys
    y1 = pow(G, x1) % P
    y2 = pow(G, x2) % P

    # Generate Secret Keys
    k1 = pow(y2, x1) % P
    k2 = pow(y1, x2) % P

    return k1.to_bytes(16, 'big'), k2.to_bytes(16, 'big')

if __name__ == "__main__":
    # Diffie-Hellman parameters
    #P = int(input("Enter P: "))  # A large prime number
    #G = int(input(f"Enter a primitive root of {P}: "))
    
    

    # Perform Diffie-Hellman key exchange
    shared_key1, shared_key2 = dh_key_exchange()
    
    # Encrypt and send a message from User 1 to User 2
    #key1 = generate_key()
    plaintext = b"Hello from User 1 to User 2"
    ciphertext = encrypt(plaintext, shared_key1)

    # Decrypt the message at User 2
    decrypted_text = decrypt(ciphertext, shared_key2)

    print("Ciphertext:", base64.b64encode(ciphertext))
    print("Decrypted Text:", decrypted_text.decode())

    # Encrypt and send a message from User 2 to User 1
    #key2 = generate_key()
    plaintext = b"Hello from User 2 to User 1"
    ciphertext = encrypt(plaintext, shared_key1)

    # Decrypt the message at User 1
    decrypted_text = decrypt(ciphertext, shared_key2)

    print("Ciphertext:", base64.b64encode(ciphertext))
    print("Decrypted Text:", decrypted_text.decode())


Ciphertext: b'320qgPMKSGYgeXy5bOLxxVr/kF0WRZ2EL1Mnza/DQyZU2/BReL4M7dOotQ=='
Decrypted Text: Hello from User 1 to User 2
Ciphertext: b'DOlXLtH6B7/RmNKXP4WdVB8TaXGUv4rYBljVQXNsCiSb/7bBN/M99BkcDw=='
Decrypted Text: Hello from User 2 to User 1
